In [1]:
%cd /content

!unzip "/content/drive/MyDrive/rgb.zip" -d dataset_raw


Streaming output truncated to the last 5000 lines.
  inflating: dataset_raw/seg/Town03_2_0.png  
  inflating: dataset_raw/seg/Town03_2_1.png  
  inflating: dataset_raw/seg/Town03_2_10.png  
  inflating: dataset_raw/seg/Town03_2_11.png  
  inflating: dataset_raw/seg/Town03_2_12.png  
  inflating: dataset_raw/seg/Town03_2_13.png  
  inflating: dataset_raw/seg/Town03_2_14.png  
  inflating: dataset_raw/seg/Town03_2_15.png  
  inflating: dataset_raw/seg/Town03_2_16.png  
  inflating: dataset_raw/seg/Town03_2_17.png  
  inflating: dataset_raw/seg/Town03_2_18.png  
  inflating: dataset_raw/seg/Town03_2_19.png  
  inflating: dataset_raw/seg/Town03_2_2.png  
  inflating: dataset_raw/seg/Town03_2_20.png  
  inflating: dataset_raw/seg/Town03_2_21.png  
  inflating: dataset_raw/seg/Town03_2_22.png  
  inflating: dataset_raw/seg/Town03_2_23.png  
  inflating: dataset_raw/seg/Town03_2_24.png  
  inflating: dataset_raw/seg/Town03_2_25.png  
  inflating: dataset_raw/seg/Town03_2_26.png  
  inflating:

In [2]:
import os
import cv2
import numpy as np

IMAGES_DIR = "dataset_raw/rgb"
MASKS_DIR = "dataset_raw/seg"
LABELS_DIR = "dataset_raw/labels"

os.makedirs(LABELS_DIR, exist_ok=True)

LANE_CLASS = 0  # one class only

def create_labels():
    img_files = sorted([f for f in os.listdir(IMAGES_DIR) if f.endswith(".png")])

    for file in img_files:
        img_path = os.path.join(IMAGES_DIR, file)
        mask_path = os.path.join(MASKS_DIR, file)

        # load RGB to know dimensions
        img = cv2.imread(img_path)
        h, w = img.shape[:2]

        # load mask (grayscale)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

        # binary mask (255 = lane, 0 = background)
        _, binary = cv2.threshold(mask, 1, 255, cv2.THRESH_BINARY)

        # find all lane blobs
        contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        label_lines = []

        for cnt in contours:
            x, y, bw, bh = cv2.boundingRect(cnt)

            # skip tiny noise
            if bw < 5 or bh < 5:
                continue

            # convert to YOLO format
            x_center = (x + bw / 2) / w
            y_center = (y + bh / 2) / h
            width = bw / w
            height = bh / h

            label_lines.append(f"{LANE_CLASS} {x_center} {y_center} {width} {height}")

        # save label file
        label_path = os.path.join(LABELS_DIR, file.replace(".png", ".txt"))
        with open(label_path, "w") as f:
            f.write("\n".join(label_lines))

        print(f"Labeled: {file} → {len(label_lines)} boxes")

if __name__ == "__main__":
    create_labels()


Streaming output truncated to the last 5000 lines.
Labeled: Town03_2_0.png → 6 boxes
Labeled: Town03_2_1.png → 4 boxes
Labeled: Town03_2_10.png → 6 boxes
Labeled: Town03_2_11.png → 6 boxes
Labeled: Town03_2_12.png → 6 boxes
Labeled: Town03_2_13.png → 5 boxes
Labeled: Town03_2_14.png → 10 boxes
Labeled: Town03_2_15.png → 13 boxes
Labeled: Town03_2_16.png → 11 boxes
Labeled: Town03_2_17.png → 9 boxes
Labeled: Town03_2_18.png → 9 boxes
Labeled: Town03_2_19.png → 9 boxes
Labeled: Town03_2_2.png → 4 boxes
Labeled: Town03_2_20.png → 8 boxes
Labeled: Town03_2_21.png → 9 boxes
Labeled: Town03_2_22.png → 9 boxes
Labeled: Town03_2_23.png → 13 boxes
Labeled: Town03_2_24.png → 14 boxes
Labeled: Town03_2_25.png → 14 boxes
Labeled: Town03_2_26.png → 9 boxes
Labeled: Town03_2_27.png → 5 boxes
Labeled: Town03_2_28.png → 4 boxes
Labeled: Town03_2_29.png → 4 boxes
Labeled: Town03_2_3.png → 4 boxes
Labeled: Town03_2_30.png → 4 boxes
Labeled: Town03_2_31.png → 3 boxes
Labeled: Town03_2_32.png → 4 boxes
La

In [3]:
import os
import random
import shutil

# paths
RGB_DIR     = "dataset_raw/rgb"
LABELS_DIR  = "dataset_raw/labels"
IMAGES_OUT  = "dataset/images"
LABELS_OUT  = "dataset/labels"  # to not overwrite original

train_ratio = 0.8  # 80% train, 20% val

train_img_dir = os.path.join(IMAGES_OUT, "train")
val_img_dir   = os.path.join(IMAGES_OUT, "val")
train_lbl_dir = os.path.join(LABELS_OUT, "train")
val_lbl_dir   = os.path.join(LABELS_OUT, "val")

os.makedirs(train_img_dir, exist_ok=True)
os.makedirs(val_img_dir,   exist_ok=True)
os.makedirs(train_lbl_dir, exist_ok=True)
os.makedirs(val_lbl_dir,   exist_ok=True)

# all pngs
images = [f for f in os.listdir(RGB_DIR) if f.lower().endswith(".png")]
images.sort()
random.shuffle(images)

split_idx = int(len(images) * train_ratio)
train_files = images[:split_idx]
val_files   = images[split_idx:]

def move_files(file_list, img_dest, lbl_dest):
    for f in file_list:
        img_src = os.path.join(RGB_DIR, f)
        lbl_src = os.path.join(LABELS_DIR, f.replace(".png", ".txt"))

        img_dst = os.path.join(img_dest, f)
        lbl_dst = os.path.join(lbl_dest, f.replace(".png", ".txt"))

        if not os.path.exists(lbl_src):
            print(f"Label missing for {f}, skipping.")
            continue

        shutil.copy2(img_src, img_dst)
        shutil.copy2(lbl_src, lbl_dst)

    print(f"Copied {len(file_list)} images to {img_dest}")

move_files(train_files, train_img_dir, train_lbl_dir)
move_files(val_files,   val_img_dir,   val_lbl_dir)
print("Done splitting.")


Copied 6720 images to dataset/images/train
Copied 1680 images to dataset/images/val
Done splitting.


In [4]:
# Where your dataset is in Colab (after unzipping & splitting)
DATASET_ROOT = "/content/dataset"

# Where to save ALL training outputs (weights, results.csv, plots) on Drive
PROJECT_DIR = "/content/drive/MyDrive/carla_lane_yolo"


In [5]:
yaml_text = f"""
path: {DATASET_ROOT}

train: images/train
val: images/val

names:
  0: lane_marking
"""

with open("carla_lane.yaml", "w") as f:
    f.write(yaml_text)

!cat carla_lane.yaml



path: /content/dataset

train: images/train
val: images/val

names:
  0: lane_marking


In [6]:
!pip install -q ultralytics
from ultralytics import YOLO


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
!yolo detect train \
  data=carla_lane.yaml \
  model=yolov8n.pt \
  epochs=40 \
  imgsz=512 \
  batch=32 \
  device=0 \
  project="$PROJECT_DIR" \
  name="exp_t4" \
  exist_ok=True


Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=carla_lane.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=exp_t4, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=

In [9]:
!yolo detect predict \
  model=/content/drive/MyDrive/carla_lane_yolo/exp_t4/weights/best.pt \
  source=/content/dataset/images/val \
  conf=0.3 \
  imgsz=640 \
  device=0 \
  save=True \
  project=/content/drive/MyDrive/carla_lane_yolo \
  name=exp_t4_preds \
  exist_ok=True


Ultralytics 8.3.229 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1680 /content/dataset/images/val/Town01_0_1.png: 480x640 2 lane_markings, 69.0ms
image 2/1680 /content/dataset/images/val/Town01_0_14.png: 480x640 2 lane_markings, 6.0ms
image 3/1680 /content/dataset/images/val/Town01_0_23.png: 480x640 1 lane_marking, 6.2ms
image 4/1680 /content/dataset/images/val/Town01_0_24.png: 480x640 (no detections), 5.9ms
image 5/1680 /content/dataset/images/val/Town01_0_25.png: 480x640 (no detections), 6.0ms
image 6/1680 /content/dataset/images/val/Town01_0_26.png: 480x640 1 lane_marking, 7.8ms
image 7/1680 /content/dataset/images/val/Town01_0_3.png: 480x640 2 lane_markings, 6.5ms
image 8/1680 /content/dataset/images/val/Town01_0_30.png: 480x640 2 lane_markings, 8.2ms
image 9/1680 /content/dataset/images/val/Town01_0_33.png: 480x640 (no detections), 5.9ms
image 10/1680 /content/dataset/images/